In [29]:
import re
import numpy as np
import heapq
from functools import reduce,cache
from operator import mul
from bisect import insort
from collections import deque,defaultdict
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
"""

### Part 1
I'm properly sick of 2d grid

In [101]:
class RaceRace:
    def __init__(self,txt):
        self.grid = txt.splitlines()                
        self.obs= set()
        self.positions = set()
        self.dirs = [1,-1,1j,-1j]
        self.row = len(self.grid)
        self.col = len(self.grid[0])
        # self.cheat_dirs = [-1+1j,2j,1+1j,2]

        for x in range(self.row):
            for y in range(self.col):
                # print(i,'',j)
                c=self.grid[x][y]
                self.positions = x+y*1j
                if c == '#':
                    self.obs.add(x+y*1j)
                elif c == 'S':
                    self.start = x+y*1j
                elif c == 'E':
                    self.end = x+y*1j
        self.distances = self.get_shortest_dists()
        
    def get_shortest_dists(self):
        open = deque()
        open.append(self.start)
        distances = defaultdict(lambda: -1)
        distances[self.start] = 0
        while open:
            current = open.popleft()
            for dir in self.dirs:
                child = current + dir
                # in bounds
                if 0<=child.real<self.row and 0<=child.imag<self.col:
                    # not #
                    if child in self.obs: continue
                    # distances[child] != -1, it's been checked
                    if distances[child] != -1: continue
                    distances[child] = distances[current]+1
                    open.append(child)
        return distances
    
    def check_cheats_deprecated(self):
        cheat_dict = defaultdict(lambda:0)
        # cheat_count = 0
        for point in self.distances:
            #  L
            # X#D
            # #L   <- all places to check
            # D
            for dir in [1,1j]:
                obs = point + dir
                if obs in self.obs:
                    # d = point + dir # direct forward
                    # l = point + dir*(1+1j) # forward left <- after attempt, terns out is unnecessary
                    candidate = obs + dir
                    # within bounds
                    if 0<=candidate.real<self.row and 0<=candidate.imag<self.col:
                        if candidate in self.obs: continue
                        # # Record all cheats
                        cheat_dict[abs(self.distances[candidate]-self.distances[point])-2] += 1
                        # # check if abs diff >= 102
        return cheat_dict # cheat_count

    def check_cheats(self):
        cheat_dict = defaultdict(lambda:0)
        # cheat_count = 0
        for point in self.distances:
            for r in range(2,21): # draw this like tree rings
                for a in range(r+1): # (0,1,..r)
                    b = r-a
                    for candidate_dir in {a+b*1j,a-b*1j,-a+b*1j,-a-b*1j}:
                        # within bounds
                        candidate = point + candidate_dir
                        if 0<=candidate.real<self.row and 0<=candidate.imag<self.col:
                            if candidate in self.obs: continue
                            # # Record all cheats
                            cheat_dict[(self.distances[candidate]-self.distances[point])-r] += 1
                            # # check if abs diff >= 102
                            # if abs(self.distances[candidate]-self.distances[point]-2) == 2:
                            #     cheat_count+=1
        return cheat_dict # cheat_count


res = RaceRace(input)
cheat_dict = res.check_cheats_deprecated()
print("part 1:",sum([v for k,v in cheat_dict.items() if k >= 100 ])) # 0.2s

# cheat_dict = res.check_cheats()
# print("part 2:",sum([v for k,v in cheat_dict.items() if k >= 100 ])) # 9.0s



part 1: 1463


### Part 1
I was really busy this morning + sick so I'm only doing AOC now. Today's challenge is to just redo what we did a hundred times already... BFS + 2d grid for shorteest path to the start. Given that we know there's only one path we can figure out the time space by doing a little subtraction +2. Only two directions of a point are considered, but both directions are counted (for example, if a-b and b-a both have an absolute value >= 100. however the checks won't have repeated due to only checking RD and R directions)

### Part 2
The scope is slightly larger and so did the searchspace. Current uses the following loops:

In [ ]:
for point in self.distances:
    for r in range(2,21): # draw this like tree rings
        for a in range(r+1): # (0,1,..r)
            b = r-a

Essentially, for every `r`, will draw a diamond with radius `r` around `point`. For example `r=2`:

```
...X...
....X..
...P.X.
.......
```
Where a in `range(0,2)` and b in `range(0,2,-1)`, drawing an diagonal line. alternating between directions will draw this:

```
...X...
..X.X..
.X.P.X.
..X.X..
...X...
```
And then it'd use similar logic to part 1. the only problem with this method is that it's really slow... But oh well I want to enjoy my friday still so there we go.